In [34]:
import pandas as pd             
import numpy as np

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor

from aux_fun import evaluate

import json

In [35]:
df = pd.read_csv('./data/dataset_imputed.csv')

In [36]:
y = df['NumberOfSales']

In [37]:
X = df.drop(df[['NumberOfSales', 'WindDirDegrees']], axis=1)

# Train-Test Split

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model 

In [39]:
#parameters
params_dict = {'n_estimators' : 15}

In [40]:
rf = RandomForestRegressor(**params_dict)

In [41]:
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=15, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [42]:
X_train.columns

Index(['StoreID', 'IsHoliday', 'IsOpen', 'HasPromotions', 'NearestCompetitor',
       'Region', 'NumberOfCustomers', 'Region_AreaKM2', 'Region_GDP',
       'Region_PopulationK', 'CloudCover', 'Max_Dew_PointC', 'Max_Humidity',
       'Max_Sea_Level_PressurehPa', 'Max_TemperatureC', 'Max_VisibilityKm',
       'Max_Wind_SpeedKm_h', 'Mean_Dew_PointC', 'Mean_Humidity',
       'Mean_Sea_Level_PressurehPa', 'Mean_TemperatureC', 'Mean_VisibilityKm',
       'Mean_Wind_SpeedKm_h', 'Min_Dew_PointC', 'Min_Humidity',
       'Min_Sea_Level_PressurehPa', 'Min_TemperatureC', 'Min_VisibilitykM',
       'Precipitationmm', 'Clear', 'Fog', 'Hail', 'Rain', 'Snow',
       'Thunderstorm', 'Hyper_Market', 'Shopping_Center', 'Standard_Market',
       'Super_Market', 'General', 'With_Fish_Department',
       'With_Non-Food_Department', 'Day', 'Month', 'Year'],
      dtype='object')

In [43]:
y_pred = rf.predict(X_test)

# Creation of the submission 

In [44]:
result = pd.DataFrame(X_test['StoreID'])
result['Month'] = X_test['Month']
result['NumberOfSales'] = y_pred
#Group by Month
result =result.groupby(['StoreID','Month'], as_index=False)['NumberOfSales'].sum()

In [45]:
X_test['NumberOfSales']= y_test

/Users/dbc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [46]:
evaluation =evaluate(X_test,result)
evaluation

KeyError: 'Date'

In [ ]:
r2 = r2_score(y_test, y_pred)
r2

In [ ]:
#r2 on grouped data
test =X_test.groupby(['StoreID','Month'], as_index=False)['NumberOfSales'].sum()
r2grouped = r2_score(test['NumberOfSales'], result['NumberOfSales'])
r2grouped

In [ ]:
model_result={'method': 'Random Forest',
       'parameters':params_dict,
       'evaluation': evaluation, 
       'r2':r2,
       'r2_grouped':r2grouped,
       'notes': 'None'
    }

In [ ]:
with open('results.json', 'a') as fp:
    json.dump(model_result, fp, indent=2)
    fp.write('\n')